In [1]:
from diffusers import DiffusionPipeline
pipeline = DiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-2-1")
model_id = "stabilityai/stable-diffusion-2-1"
# Load the tokenizer and text encoder to tokenize and encode the text.
tokenizer = pipeline.tokenizer
text_encoder = pipeline.text_encoder
del pipeline
text_encoder = text_encoder.to('cuda')

text_encoder/pytorch_model.fp16.safetensors not found


Fetching 22 files:   0%|          | 0/22 [00:00<?, ?it/s]

In [3]:
prompt = "a robot working at a computer"

In [4]:
?tokenizer

Signature:     
tokenizer(
    text: Union[str, List[str], List[List[str]]] = None,
    text_pair: Union[str, List[str], List[List[str]], NoneType] = None,
    text_target: Union[str, List[str], List[List[str]]] = None,
    text_pair_target: Union[str, List[str], List[List[str]], NoneType] = None,
    add_special_tokens: bool = True,
    padding: Union[bool, str, transformers.utils.generic.PaddingStrategy] = False,
    truncation: Union[bool, str, transformers.tokenization_utils_base.TruncationStrategy] = None,
    max_length: Optional[int] = None,
    stride: int = 0,
    is_split_into_words: bool = False,
    pad_to_multiple_of: Optional[int] = None,
    return_tensors: Union[str, transformers.utils.generic.TensorType, NoneType] = None,
    return_token_type_ids: Optional[bool] = None,
    return_attention_mask: Optional[bool] = None,
    return_overflowing_tokens: bool = False,
    return_special_tokens_mask: bool = False,
    return_offsets_mapping: bool = False,
    return_length:

In [5]:
tokenizer.model_max_length

77

In [13]:
# Turn the text into a sequnce of tokens:
#text_input = tokenizer(prompt, padding=False, max_length=tokenizer.model_max_length, truncation=True, return_tensors="pt")
text_input = tokenizer(prompt, padding="max_length", max_length=tokenizer.model_max_length, truncation=True, return_tensors="pt")
text_input['input_ids'][0] # View the tokens

tensor([49406,   320,  8797,  1699,   536,   320, 11639,   652, 49407,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0])

In [14]:
# See the individual tokens
for t in text_input['input_ids'][0][:15]:
    print(t, tokenizer.decoder.get(int(t)))

tensor(49406) <|startoftext|>
tensor(320) a</w>
tensor(8797) robot</w>
tensor(1699) working</w>
tensor(536) at</w>
tensor(320) a</w>
tensor(11639) compu
tensor(652) ter</w>
tensor(49407) <|endoftext|>
tensor(0) !
tensor(0) !
tensor(0) !
tensor(0) !
tensor(0) !
tensor(0) !


In [15]:
# Grab the output embeddings
output_embeddings = text_encoder(text_input.input_ids.to('cuda'))[0]
print('Shape:', output_embeddings.shape)
output_embeddings

Shape: torch.Size([1, 77, 1024])


tensor([[[-0.3134, -0.4476, -0.0082,  ...,  0.2542, -0.0324, -0.2960],
         [ 0.1996, -1.6936, -0.8946,  ...,  0.4658, -0.0956, -2.1484],
         [ 0.7975, -0.6343, -1.5664,  ..., -0.4636,  0.0522, -0.7181],
         ...,
         [ 0.1910, -1.6957,  0.4581,  ...,  0.0701, -0.3644,  0.4921],
         [ 0.2270, -1.6606,  0.4586,  ..., -0.2156, -0.4088,  0.6127],
         [ 0.1552, -2.6756, -0.1644,  ...,  0.1500, -0.2731,  1.0431]]],
       device='cuda:0', grad_fn=<NativeLayerNormBackward0>)